In [1]:
from client import start_client
import numpy as np

client,w=start_client(5)
client

<Client: 'tcp://127.0.0.1:39423' processes=5 threads=5, memory=33.69 GB>


Client Scheduler: tcp://127.0.0.1:39423 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 33.69 GB


In [2]:
from main_func import main
#feed client,workers,num of samples and num of features
main(client,w,240,4)

coo ([array([-15.78093963,  -7.96381499,  26.54580668,  -9.93710109]), -9.81279338689866], 1, 1, [1])
Accuracy: 83.333333
coo ([array([ -3.64924111,  13.35352759,  15.30306978, -19.5662096 ]), -9.842139046673337], 1, 15, [15])
Accuracy: 73.333333
coo ([array([-13.91724482, -10.60407132,  21.25116164,  -4.69422817]), -18.374562254864465], 1, 1, [1])
Accuracy: 81.666667
coo ([array([-13.91724482, -10.60407132,  21.25116164,  -4.69422817]), -18.374562254864465], 1, 1, [])
Accuracy: 81.666667

End of chunks.For now on E will stay the same!
['finished', 'finished', 'finished', 'finished', 'finished']
finished
coo ([array([-13.91724482, -10.60407132,  21.25116164,  -4.69422817]), -18.374562254864465], 1, 1, [])
Accuracy: 81.666667
